## Multi-fidelity Bayesian Optimization

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math

import pandas as pd
import torch

from botorch.test_functions.multi_fidelity import AugmentedHartmann
problem = AugmentedHartmann()

def test_function(input_dict):
    x = torch.tensor([input_dict[f"x{i}"] for i in range(1,7)])
    s = torch.tensor(input_dict["s"]).unsqueeze(0)
    return {"f": problem(torch.cat((x,s))).numpy()}


# define vocs
from xopt import VOCS
vocs = VOCS(
    variables={
        "x1": [0, 1],
        "x2": [0, 1],
        "x3": [0, 1],
        "x4": [0, 1],
        "x5": [0, 1],
        "x6": [0, 1],
    },
    objectives={"f": "MINIMIZE"},
)

In [2]:
# create xopt object
from xopt.generators import MultiFidelityBayesianGenerator
from xopt import Evaluator, Xopt

# get and modify default generator options
options = MultiFidelityBayesianGenerator.default_options()

# set the base cost of evaluating the function
options.acq.base_cost = 5.0
options.acq.fidelities = [0.5,0.75,1.0]

generator = MultiFidelityBayesianGenerator(vocs, options=options)
evaluator = Evaluator(function=test_function)

X = Xopt(vocs=vocs, generator=generator, evaluator=evaluator)
X.options.strict = True

ValueError: "MultiFidelityAcqOptions" object has no field "base_cost"

In [ ]:
# evaluate random initial points at mixed fidelities to seed optimization and view data
X.evaluate_data(
    pd.DataFrame(np.random.rand(5,7), columns=X.vocs.variable_names + ["s"])
)
X.data

In [ ]:
# get the total cost of previous observations (base_cost + s for each measurement)
X.generator.calculate_total_cost()

In [ ]:
# run optimization until the cost budget is exhausted
budget = 10
while X.generator.calculate_total_cost() < budget:
    print("step")
    X.step()

In [ ]:
X.data

In [ ]:
X.generator.calculate_total_cost()